In [10]:
trafficname = "prv1"
H = 1
errortolerance = 0.01

if trafficname.startswith("cluster"):
    numinterval = 48
    scaledown = 100000
else:
    numinterval = 8
    scaledown = 10000000
routingnamearr = ["ecmp","su2","su3","32disjoint"]
outputfile = f"/home/annzhou/DRing/src/emp/datacentre/computerouting2/output/votespatial_{trafficname}_{H}_{errortolerance}"
inputfile = f"/home/annzhou/DRing/src/emp/datacentre/computerouting2/output/spatial_dring_{trafficname}_-1_0_64_d{scaledown}"

datadict = dict()
for routingname in routingnamearr:
    datadict[routingname] = dict()
with open(inputfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        routingname = tokens[0]
        interval = int(tokens[1])
        throughput = float(tokens[2])
        datadict[routingname][interval] = throughput


# from chatgpt
def find_max_indices(numbers, tolerance):
    # Find the maximum value in the list
    max_value = max(numbers)
    
    # Find all indices where the value is within the tolerance of the maximum value
    indices = [i for i, value in enumerate(numbers) if abs(value - max_value)/max_value <= tolerance]
    
    return indices


vote_per_interval = list()
for interval in range(numinterval):
    throughputlist = list()
    for routingname in routingnamearr:
        throughputlist.append(datadict[routingname][interval])
    vote_indices = find_max_indices(throughputlist,errortolerance)
    vote_per_interval.append(vote_indices)

with open(outputfile,'w') as f:
    for solveinterval in range(H,numinterval):
        totalvote = [0,0,0,0]
        for computeinterval in range(solveinterval-H,solveinterval):
            myvotes = vote_per_interval[computeinterval]
            for myvote in myvotes:
                totalvote[myvote] += 1
        maxvotes = find_max_indices(totalvote,0)
        for maxvote in maxvotes:
            f.write(f"{solveinterval},{maxvote}\n")